In [1]:
import argparse
import logging
import math
import os
import random

import datasets
import nltk
import numpy as np
import torch
from datasets import load_dataset, load_metric
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    SchedulerType,
    get_scheduler,
    set_seed,
    XLMProphetNetTokenizer, XLMProphetNetForConditionalGeneration, XLMProphetNetConfig
)
from transformers.file_utils import is_offline_mode
from transformers.utils.versions import require_version
from preprocess_data import load_xglue
import pickle
import sacrebleu

In [2]:
os.environ ["CUDA_VISIBLE_DEVICES"] = '0'

In [3]:
class Args():
    model_name_or_path = 'microsoft/xprophetnet-large-wiki100-cased-xglue-ntg'
    cache_dir = "/home/work/xiaoyu/ckpt/xprophtnet_ntg"
    use_fast_tokenizer = True
    
args = Args()

In [22]:
config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=args.use_fast_tokenizer, cache_dir=args.cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name_or_path, from_tf=bool(".ckpt" in args.model_name_or_path), config=config, cache_dir=args.cache_dir)

In [23]:
model.resize_token_embeddings(len(tokenizer))

Embedding(250012, 1024, padding_idx=0)

In [74]:
fr_sent = "Le procès d'un Turc de 17 ans qui avait agressé en janvier 2016 à la machette un enseignant d'une école juive de Marseille portant une kippa, s'ouvre mercredi devant le tribunal pour enfants (TPE) de Paris. Le jeune, qui avait 15 ans au moment des faits, et s'était revendiqué de l'Etat islamique, encourt jusqu'à vingt ans de réclusion criminelle, apprend-on de source judiciaire. Il est poursuivi pour entreprise terroriste individuelle et tentative d'assassinat en relation avec une entreprise terroriste, avec la circonstance aggravante de l'antisémitisme, précise-t-on. Il s'agit du premier procès terroriste criminel devant le TPE de Paris, qui n'a jusque-là été amené à juger que des affaires djihadistes délictuelles. D'après Fabrice Labi, avocat de la victime, l'enseignant arrive à ce procès avec énormément de questions. Il a juste besoin de comprendre pourquoi ce garçon en est arrivé là et pourquoi il s'en est pris à lui, a-t-il dit à Reuters. De comprendre comment ce garçon a pu se radicaliser au point de vouloir donner la mort, a-t-il ajouté. L'agresseur, actuellement incarcéré, a manifesté des regrets en prison, assure Fabrice Labi. Mais sa volonté de rédemption est arrivée très tard, trois-quatre mois avant le procès, dit-il, ajoutant qu'il ne s'est jamais excusé. LE TRIBUNAL POUR ENFANTS SE PRÉPARE DEPUIS PLUSIEURS MOIS L'enseignant, très fortement touché par les faits, n'a toujours pas repris son travail, précise-t-il. L'agression avait suscité une vive émotion au sein de la communauté juive. Evoquant une situation exceptionnelle, le président du Consistoire israélite de Marseille avait notamment recommandé aux fidèles de ne pas porter la kippa provisoirement, une prise de position critiquée par le président du Conseil représentatif des institutions juives de France (CRIF) et le grand rabbin de France pour qui cela revenait à céder à lEtat islamique. Depuis plusieurs mois, le tribunal pour enfants de Paris se prépare à une véritable déferlante de procès djihadistes, après la mise en examen de plusieurs mineurs soupçonnés davoir projeté des attentats. Dès septembre 2016, le président du tribunal a créé un groupe de huit juges dédiés sur 14. Spécialement formés, ils sont les seuls à siéger pour les dossiers djihadistes. Une dizaine dassesseurs (membres de la société civile qui composent le tribunal avec le juge), se sont également constitués en pôle spécialisé. En décembre dernier, une trentaine de juges, assesseurs et éducateurs ont par ailleurs reçu une formation spécifique pour faire face à ce phénomène nouveau mais persistant. En décembre dernier, 50 mineurs, dont 14 filles, étaient mis en examen dans des dossiers djihadistes. (Chine Labbé à Paris et Jean-François Rosnoblet à Marseille, édité par Yves Clarisse)"
#inputs = tokenizer([fr_sent], padding=True, max_length=256, return_tensors='pt')
inputs = tokenizer([fr_sent], padding=False, max_length=1024, truncation=True, return_tensors='pt')

In [73]:
# fr_sent = "Vice-présidente de l'Assemblée nationale, la macroniste Carole Bureau-Bonnard était chargée mardi après-midi d'animer la séance d'examen du projet de loi «confiance dans l'action publique». C'était sans compter son manque d'expérience... «C'est le chantier!» Carole Bureau-Bonnard, nouvelle députée macroniste de l'Oise, devenue vice-présidente de l'Assemblée nationale dans la foulée, a connu mardi après-midi une séance éprouvante. Chargée d'animer l'examen du projet de loi «confiance dans l'action publique» , la primo-députée a sombré. Malgré l'aide d'un fonctionnaire de l'Assemblée nationale qui lui soufflait ce qu'elle devait faire, Carole Bureau-Bonnard a dû quitter le perchoir, le président François de Rugy ayant été rappelé en urgence... Avant cela, des députés de toutes tendances ont fustigé le manque de tenue de la séance. C'est d'abord le socialiste Olivier Dussopt qui a laissé entendre que la macroniste faisait planer une pression sur les députés du parti majoritaire, invités à voter à mains levées divers amendements: «Les députés lèvent la main ou ne la lèvent pas. Ils peuvent faire le choix de ne pas participer à un vote. Ils peuvent faire le choix de s'abstenir, de se prononcer, mal à l'aise par une disposition ou par conviction du fait de ne pas vouloir adhérer à tel ou tel article. Vous avez appelé le vote deux fois, presque trois (sur un même amendement, NDLR). Soit cette situation ne se reproduit pas. Soit au nom de mon groupe, je serai amené à demander un scrutin public (électronique, NDLR) sur chacun des amendements déposés», a lancé le parlementaire provoquant les applaudissements... de la droite. «Est-ce qu'on peut faire une suspension de séance?» Et Alexis Corbière (France Insoumise) de se faire entendre: «Il faut que cette assemblée puisse avoir pleinement confiance (dans les) votes, visiblement ce n'est pas le cas». Face à la pagaille, Carole Bureau-Bonnard n'a pas eu d'autre choix que de se tourner vers un fonctionnaire de l'hémicycle: «Est-ce qu'on peut faire une suspension de séance? C'est le chantier». «La phase d'apprentissage, ça va bien!» La reprise n'a pas été plus sereine: «Force est de constater que nous ne savons pas si l'amendement est voté ou pas. Ça flotte! Et le débat n'avance pas», s'étrangle alors Philippe Vigier (Constructifs). Olivier Dussopt est ensuite revenu à la charge: «La confusion dans laquelle nous travaillons depuis cette nuit, encore cet après-midi, fait que notre assemblée s'approche assez dangereusement de la ligne qui sépare le principe de clarté et de sincérité des débats, de la ligne de l'irrespect de ce principe. Si la séance ne retrouve pas un déroulé plus serein, c'est la constitutionnalité du texte et la validation par le Conseil constitutionnel que vous mettez en péril», a-t-il mis en garde. La grogne a ensuite gagné la majorité quand Jean-Louis Bourlanges (MoDem) a demandé à prendre la parole. Face au refus de Carole Bureau-Bonnard de la lui donner, il a furieusement quitté l'hémicycle. Sans que l'ensemble de son propos soit intelligible, la vice-présidente de l'Assemblée grommelle: «Ben, il part, hein, tant pis... Ben c'est bien comme ça je peux....» «Vraiment, ça devient pénible!», a chargé André Chassaigne (Communiste). «Vous menez ces débats d'une façon incompréhensible», a-t-il asséné. Philippe Gosselin des Républicains: «La phase d'apprentissage, ça va bien! Ici on n'est pas dans une phase d'apprentissage, on fabrique la loi». «Donc il va falloir peut-être appliquer le règlement», a-t-il poursuivi en se lançant dans une explication de la procédure à suivre. Seul le jeune macroniste Saché Houlié, lui aussi vice-président de l'Assemblée, a pris la défense de Carole Bureau-Bonnard. «Je voulais vous féliciter Madame la présidente parce que je sais à quel point c'est difficile de mener les débats dans cet hémicycle. Je sais à quel point vous le faites correctement». Un encouragement insuffisant. François de Rugy a fait son apparition quelques minutes après pour mettre fin au naufrage..."
fr_sent = "Vice-présidente de l'Assemblée nationale, la macroniste Carole Bureau-Bonnard était chargée mardi après-midi d'animer la séance d'examen du projet de loi «confiance dans l'action publique». C'était sa"
#inputs = tokenizer([fr_sent], padding=False, max_length=1024, truncation=True, return_tensors='pt')
inputs = tokenizer([fr_sent], padding="max_length", max_length=512, truncation=True, return_tensors='pt')
#print("inputs", inputs)

In [75]:
#summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=128)

In [76]:
print(summary_ids)

tensor([[    2,   647,  9893, 10413, 17158, 43009,    24, 13636,  2973,   152,
            19,  9953,    21,  1186, 23425,   542,   395, 21918,    19,  7281,
             2]])


In [77]:
tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

['Le premier procès terroriste criminel de France a eu lieu au TPE de Paris']

In [10]:
accelerator = Accelerator()

In [11]:
model = accelerator.prepare(model)
print(next(model.parameters()).device) 

cuda:0


In [18]:
inputs['input_ids'] = accelerator.prepare(inputs['input_ids'])
print(inputs['input_ids'].device) 

cpu


In [19]:
inputs['input_ids'], inputs['attention_mask'] = inputs['input_ids'].to("cuda:0"), inputs['attention_mask'].to("cuda:0")
print(inputs['input_ids'].device) 
print(next(model.parameters()).device) 

cuda:0
cuda:0


In [20]:
generated_tokens = accelerator.unwrap_model(model).generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], num_beams=4, max_length=128)

In [21]:
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Les voyageurs qui réservent leur voyage sur leur smartphone semblent avoir les mêmes préférences']